In [1064]:
import pandas as pd
import numpy as np
import plotly.express as px
import klib as kl

# impute missing values with iterative imputer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [1065]:
products_data = pd.read_csv('data/regression_clean_data.csv')



In [1066]:
data = pd.read_csv('data/site_changes_cleaned.csv')



# create a date column from month and year and add day as 1
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))
data = kl.clean_column_names(data)

data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"incentive increase, apr decrease",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [1067]:
data.groupby('bank')['incentive'].sum()

bank
abanca           1500
banc sabadell    1050
bankinter           0
bbva                0
evobanco            0
myinvestor          0
n26                 0
openbank            0
revolut             0
santander         850
Name: incentive, dtype: int64

In [1068]:
data.groupby('bank')['apr'].sum()

bank
abanca            4.00
banc sabadell    14.00
bankinter        15.00
bbva              6.00
evobanco          7.47
myinvestor        7.00
n26               2.26
openbank          5.14
revolut           0.00
santander         0.00
Name: apr, dtype: float64

In [1069]:
data.tag.value_counts()

tag
increase incentive                  15
increase apr                        15
incentive decrease                   3
decrease apr                         3
incentive decrease, apr increase     2
incentive increase, apr decrease     1
incentive increase, apr increase     1
incentive decrease, apr decrease     1
temporary incentive                  1
Name: count, dtype: int64

In [1070]:
data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"incentive increase, apr decrease",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [1071]:
# rename tags

data.tag = data.tag.replace({'apr increase':'increase apr', 'incentive increase':'increase incentive', 'decrease incentive':'incentive decrease',  'decrease incentive ':'incentive decrease',
'incentive decrease, apr increase':'decrease incentive, increase apr',
'incentive increase, apr decrease':'increase incentive, decrease apr',
'incentive increase, apr increase':'increase incentive, increase apr',
'incentive decrease, apr decrease':'decrease incentive, decrease apr'})

       
data.tag.value_counts()

tag
increase incentive                  15
increase apr                        15
incentive decrease                   3
decrease apr                         3
decrease incentive, increase apr     2
increase incentive, decrease apr     1
increase incentive, increase apr     1
decrease incentive, decrease apr     1
temporary incentive                  1
Name: count, dtype: int64

In [1072]:
data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"increase incentive, decrease apr",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [1073]:
mapping_dict = {
    'abanca': 'abanca',
    'banc sabadell': 'banc sabadell',
    'bancosabadell': 'banc sabadell',
    'bancosantander': 'santander',
    'bancsabadell': 'banc sabadell',
    'sabadell': 'banc sabadell',
    'bank': 'bank',
    'bankinter': 'bankinter',
    'bbva': 'bbva',
    'caixabank': 'caixabank',
    'caixa_bank': 'caixabank',
    'date': 'date',
    'evobanco': 'evobanco',
    'evo_banco': 'evobanco',
    'imagin': 'imagin',
    'ing': 'ing',
    'ing-bank': 'ing',
    'ing_bank': 'ing',
    'kutxabank': 'kutxabank',
    'liberbank': 'liberbank',
    'myinvestor': 'myinvestor',
    'n26': 'n26',
    'n26.com': 'n26',
    None: None,  # handling nan (not a number)
    'openbank': 'openbank',
    'orangebank': 'orangebank',
    'pibank': 'pibank',
    'renaultbank': 'renaultbank',
    'revolut': 'revolut',
    'santander': 'santander',
    'value': 'value'
}

# map santander to bancosantander in data 
data.bank = data.bank.map(mapping_dict)
products_data.bank = products_data.bank.map(mapping_dict)



create two lists one for APR and Incentive increases. 

In [1074]:
products_data.head()

,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1.0
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1.0
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2.0
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.0,150.0,0.0,2.0
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.0,300.0,0.0,2.0


In [1075]:
# cast date to datetime
products_data['date'] = pd.to_datetime(products_data['date'])
products_data = products_data[['date', 'bank', 'web_traffic']]

data['date'] = pd.to_datetime(data['date']) 

In [1076]:
data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"increase incentive, decrease apr",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [1077]:

# attach product_data to data
data = pd.merge(products_data, data, on=['bank', 'date'], how='left')

# drop the 124 row 
data = data.drop(124)

data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))


# for some reason banks are not having the dates attaching properly to the bank data so we have to re add all of the dates to the banks

for bank in data.bank.unique():
    data.loc[data.bank == bank, 'date'] = list(products_data.date.unique())


separate the data into increase apr and increase incentive lists 


In [1078]:

data.tag = data.tag.fillna('')
apr_increase_dates = data[data.tag.str.contains('increase apr')]
incentive_increase_dates = data[data.tag.str.contains('increase incentive')]


all_dates = data.date.unique()


In [1079]:
apr_increase_dates = apr_increase_dates[['date', 'bank', 'apr']]
incentive_increase_dates = incentive_increase_dates[['date', 'bank', 'incentive']]
apr_increase_dates

,date,bank,apr
19,2023-07-01,abanca,2.00
24,2023-12-01,abanca,2.00
44,2023-07-01,n26,2.26
58,2022-08-01,banc sabadell,1.00
61,2022-11-01,banc sabadell,2.00
63,2023-01-01,banc sabadell,2.00
67,2023-05-01,banc sabadell,2.50
155,2022-04-01,openbank,1.00
167,2023-04-01,openbank,1.25
173,2023-10-01,openbank,1.76


In [1080]:
banks = data.bank.unique()

In [1081]:
# log scale web traffic
data['web_traffic'] = np.log(data['web_traffic'])
products_data['web_traffic'] = np.log(products_data['web_traffic'])

In [1082]:
control_df_avg = pd.DataFrame(data.groupby('date')['web_traffic'].mean()).reset_index()
control_df_avg['apr'] = 0
control_df_avg['incentive'] = 0

here we create a dictionary for each bank as a key and each value is a dataframe of the bank's data.

In [1083]:
# for each bank we take a subsection of the data that has no incentives 

bank_data_dict = {}

for bank in banks:
    # for each bank attach the relevant apr and incentive data
    
    apr_data = apr_increase_dates[apr_increase_dates.bank == bank].fillna(0)
    incentive_data = incentive_increase_dates[incentive_increase_dates.bank == bank].fillna(0)

    # merge the apr and incentives data with a copy of the products data specific to the bank
    temp_products_data = products_data[products_data.bank == bank].fillna(0)
    bank_data = temp_products_data.merge(apr_data, on=['date','bank'], how='left')
    bank_data = bank_data.merge(incentive_data, on=['date','bank'], how='left')
    print(bank_data.incentive.unique())

    print(bank_data.apr.value_counts())
    print(bank_data.groupby('bank')['date'].count())

    bank_data_dict[bank] = bank_data


bank_data_dict
    

    

[ nan 300.]
apr
2.0    2
Name: count, dtype: int64
bank
abanca    25
Name: date, dtype: int64
[nan]
apr
2.26    1
Name: count, dtype: int64
bank
n26    25
Name: date, dtype: int64
[ nan  50.   0. 250.]
apr
2.0    2
1.0    1
2.5    1
Name: count, dtype: int64
bank
banc sabadell    25
Name: date, dtype: int64
[nan]
Series([], Name: count, dtype: int64)
bank
ing    25
Name: date, dtype: int64
[nan  0.]
Series([], Name: count, dtype: int64)
bank
bbva    25
Name: date, dtype: int64
[nan  0.]
Series([], Name: count, dtype: int64)
bank
revolut    25
Name: date, dtype: int64
[nan]
apr
1.00    1
1.25    1
1.76    1
Name: count, dtype: int64
bank
openbank    25
Name: date, dtype: int64
[nan]
apr
1.0    2
2.0    1
Name: count, dtype: int64
bank
myinvestor    25
Name: date, dtype: int64
[nan  0.]
Series([], Name: count, dtype: int64)
bank
bankinter    25
Name: date, dtype: int64
[nan]
apr
0.06    1
2.00    1
2.50    1
2.85    1
Name: count, dtype: int64
bank
evobanco    25
Name: date, dtype: int64

{'abanca':          date    bank  web_traffic  apr  incentive
 0  2021-12-01  abanca     9.629247  NaN        NaN
 1  2022-01-01  abanca    10.649794  NaN        NaN
 2  2022-02-01  abanca    10.448661  NaN        NaN
 3  2022-03-01  abanca    10.450056  NaN        NaN
 4  2022-04-01  abanca    10.609262  NaN      300.0
 5  2022-05-01  abanca    10.301973  NaN        NaN
 6  2022-06-01  abanca    10.465598  NaN        NaN
 7  2022-07-01  abanca    11.034398  NaN        NaN
 8  2022-08-01  abanca    10.895609  NaN        NaN
 9  2022-09-01  abanca    10.839406  NaN        NaN
 10 2022-10-01  abanca    10.831057  NaN        NaN
 11 2022-11-01  abanca    10.900082  NaN        NaN
 12 2022-12-01  abanca    11.129960  NaN        NaN
 13 2023-01-01  abanca    11.189557  NaN        NaN
 14 2023-02-01  abanca    10.809217  NaN        NaN
 15 2023-03-01  abanca    10.965312  NaN        NaN
 16 2023-04-01  abanca    10.876991  NaN        NaN
 17 2023-05-01  abanca    10.966861  NaN        NaN
 1

filling na values


In [1084]:
# drop bank_x and bank_y columns
for bank in banks:
    bank_data = bank_data_dict[bank]
    #bank_data = bank_data.drop(columns=['bank_x', 'bank_y'])
    bank_data.fillna(0, inplace=True)
    bank_data_dict[bank] = bank_data

here we create two functions to determine if the event is valid and another to find consecutive dates surrounding the event date 

In [1085]:
def check_if_valid_event(bank_data, date, buffer, column):
    date_index = bank_data[bank_data.date == date].index[0]
    for i in range(1, buffer + 1):
        prev_date = bank_data.iloc[date_index - i].date
        next_date = bank_data.iloc[date_index + i].date
        prev_apr = bank_data.iloc[date_index - i][column]
        next_apr = bank_data.iloc[date_index + i][column]

        print(f"Checking date: {date}, Buffer-{i}: {prev_date} (APR: {prev_apr}), Buffer+{i}: {next_date} (APR: {next_apr})")
        
        if prev_apr != 0 or next_apr != 0:
            return False

    return True

def find_consecutive_dates(dates, buffer):
    # Calculate the required length of the sequence
    required_length = (buffer * 2) + 1
    
    # Convert dates to a sorted list of pandas Timestamps
    sorted_dates = sorted(pd.to_datetime(dates))
    
    # Iterate through the sorted dates list
    for i in range(len(sorted_dates) - required_length + 1):
        # Extract the sublist of the required length
        sublist = sorted_dates[i:i + required_length]
        
        
        # Check if the sublist is consecutive months
        if all((sublist[j + 1].year * 12 + sublist[j + 1].month) - (sublist[j].year * 12 + sublist[j].month) == 1 for j in range(len(sublist) - 1)):
            return sublist
    
    # If no such sequence is found
    return None

here we run a loop for each bank then find a control date in the 'ing' series which does not alter or change incentives or aprs.

this is for apr changes

In [1086]:


buffer = 2

# For each event date we verify that there is 1 day before and after that are not events
all_regressions_apr = {'regressions': []}

# For each bank we find an event date
for bank in banks:
    try:
        bank_data = bank_data_dict[bank]
        
        # Find dates where there is an APR increase
        apr_increase_dates = bank_data[bank_data.apr > 0]
        
        for date in apr_increase_dates.date:
            is_valid = check_if_valid_event(bank_data, date, buffer, 'apr')
            # If the date is valid we check for a control date in another bank
            if is_valid:
                bank_control = 'avg'
                bank_data_control = control_df_avg
                control_date = bank_data_control[bank_data_control.date == date]
                if not control_date.empty and control_date.apr.iloc[0] == 0 and check_if_valid_event(bank_data_control, date, buffer, 'apr'):
                    # Now we have a valid event and control date
                    # We now need to find a control period where neither bank has activity

                    # Create a list of dates that are not events for either bank or the control period
                    viable_control_dates = bank_data_control[(bank_data_control.apr == 0) & (bank_data_control.incentive == 0)].date
                    # Also remove the event dates + buffer
                    viable_control_dates = viable_control_dates[viable_control_dates != date]
                    for i in range(1, buffer + 1):
                        viable_control_dates = viable_control_dates[viable_control_dates != date + pd.DateOffset(i)]
                        viable_control_dates = viable_control_dates[viable_control_dates != date - pd.DateOffset(i)]
                    
                    # Do the same for the bank dates
                    viable_bank_dates = bank_data[(bank_data.apr == 0) & (bank_data.incentive == 0)].date
                    viable_bank_dates = viable_bank_dates[viable_bank_dates != date]
                    for i in range(1, buffer + 1):
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date + pd.DateOffset(i)]
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date - pd.DateOffset(i)]

                    # Now we have a list of viable control dates for both banks
                    # We need to find the intersection of these two lists of dates
                    viable_periods = viable_control_dates[viable_control_dates.isin(viable_bank_dates)]

                    # Now we need to find a sequence of consecutive dates that are in the viable periods and (buffer*2)+1 dates long
                    control_period = find_consecutive_dates(viable_periods, buffer)
                    print(control_period)
                    
                    # Now we create each series with the relevant data
                    event_series = bank_data[(bank_data.date >= date - pd.DateOffset(months=buffer)) & (bank_data.date <= date + pd.DateOffset(months=buffer))]
                    event_control_series = bank_data_control[(bank_data_control.date >= date - pd.DateOffset(months=buffer)) & (bank_data_control.date <= date + pd.DateOffset(months=buffer))]

                    control_event_series = bank_data[(bank_data.date >= control_period[0]) & (bank_data.date <= control_period[-1])]
                    control_series = bank_data_control[(bank_data_control.date >= control_period[0]) & (bank_data_control.date <= control_period[-1])]

                    # Now we add it to a dictionary
                    regression = {
                        'banks_for_regression': f'{bank} * {bank_control}', 
                        'bank_of_interest_event_series': event_series, 
                        'control_bank_event_series': event_control_series, 
                        'bank_of_interest_non_event_series': control_event_series, 
                        'control_bank_non_event_series': control_series
                    }
                    all_regressions_apr['regressions'].append(regression)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")

all_regressions_apr


Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0.0), Buffer+1: 2023-08-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0.0), Buffer+2: 2023-09-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0), Buffer+1: 2023-08-01 00:00:00 (APR: 0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0), Buffer+2: 2023-09-01 00:00:00 (APR: 0)
[Timestamp('2022-05-01 00:00:00'), Timestamp('2022-06-01 00:00:00'), Timestamp('2022-07-01 00:00:00'), Timestamp('2022-08-01 00:00:00'), Timestamp('2022-09-01 00:00:00')]
Error processing bank abanca: single positional indexer is out-of-bounds
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0.0), Buffer+1: 2023-08-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0.0), Buffer+2: 2023-09-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023

{'regressions': [{'banks_for_regression': 'abanca * avg',
   'bank_of_interest_event_series':          date    bank  web_traffic  apr  incentive
   17 2023-05-01  abanca    10.966861  0.0        0.0
   18 2023-06-01  abanca    11.076411  0.0        0.0
   19 2023-07-01  abanca    11.264156  2.0        0.0
   20 2023-08-01  abanca    11.415183  0.0        0.0
   21 2023-09-01  abanca    11.143055  0.0        0.0,
   'control_bank_event_series':          date  web_traffic  apr  incentive
   17 2023-05-01     9.836984    0          0
   18 2023-06-01     9.768123    0          0
   19 2023-07-01     9.885911    0          0
   20 2023-08-01     9.831852    0          0
   21 2023-09-01     9.827025    0          0,
   'bank_of_interest_non_event_series':         date    bank  web_traffic  apr  incentive
   5 2022-05-01  abanca    10.301973  0.0        0.0
   6 2022-06-01  abanca    10.465598  0.0        0.0
   7 2022-07-01  abanca    11.034398  0.0        0.0
   8 2022-08-01  abanca    10

now we do the same for incentives 

In [1087]:
# For each event date we verify that there is 1 day before and after that are not events
all_regressions_incentives = {'regressions': []}

# For each bank we find an event date
for bank in banks:
    try:
        bank_data = bank_data_dict[bank]
        
        # Find dates where there is an APR increase
        incentive_increase_dates = bank_data[bank_data.incentive > 0]
        
        for date in incentive_increase_dates.date:
            is_valid = check_if_valid_event(bank_data, date, buffer, 'incentive')
            # If the date is valid we check for a control date in another bank
            if is_valid:
                bank_control = 'avg'
                # create a column that is the average of all banks that are not the bank of interest
                bank_data_control = control_df_avg
                control_date = bank_data_control[bank_data_control.date == date]
                if not control_date.empty and control_date.incentive.iloc[0] == 0 and check_if_valid_event(bank_data_control, date, buffer, 'incentive'):
                    # Now we have a valid event and control date
                    # We now need to find a control period where neither bank has activity

                    # Create a list of dates that are not events for either bank or the control period
                    viable_control_dates = bank_data_control[(bank_data_control.incentive == 0) & (bank_data_control.incentive == 0)].date
                    # Also remove the event dates + buffer
                    viable_control_dates = viable_control_dates[viable_control_dates != date]
                    for i in range(1, buffer + 1):
                        viable_control_dates = viable_control_dates[viable_control_dates != date + pd.DateOffset(i)]
                        viable_control_dates = viable_control_dates[viable_control_dates != date - pd.DateOffset(i)]
                    
                    # Do the same for the bank dates
                    viable_bank_dates = bank_data[(bank_data.incentive == 0) & (bank_data.incentive == 0)].date
                    viable_bank_dates = viable_bank_dates[viable_bank_dates != date]
                    for i in range(1, buffer + 1):
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date + pd.DateOffset(i)]
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date - pd.DateOffset(i)]

                    # Now we have a list of viable control dates for both banks
                    # We need to find the intersection of these two lists of dates
                    viable_periods = viable_control_dates[viable_control_dates.isin(viable_bank_dates)]

                    # Now we need to find a sequence of consecutive dates that are in the viable periods and (buffer*2)+1 dates long
                    control_period = find_consecutive_dates(viable_periods, buffer)
                    print(control_period)
                    
                    # Now we create each series with the relevant data
                    event_series = bank_data[(bank_data.date >= date - pd.DateOffset(months=buffer)) & (bank_data.date <= date + pd.DateOffset(months=buffer))]
                    event_control_series = bank_data_control[(bank_data_control.date >= date - pd.DateOffset(months=buffer)) & (bank_data_control.date <= date + pd.DateOffset(months=buffer))]

                    control_event_series = bank_data[(bank_data.date >= control_period[0]) & (bank_data.date <= control_period[-1])]
                    control_series = bank_data_control[(bank_data_control.date >= control_period[0]) & (bank_data_control.date <= control_period[-1])]

                    # Now we add it to a dictionary
                    regression = {
                        'banks_for_regression': f'{bank} * {bank_control}', 
                        'bank_of_interest_event_series': event_series, 
                        'control_bank_event_series': event_control_series, 
                        'bank_of_interest_non_event_series': control_event_series, 
                        'control_bank_non_event_series': control_series
                    }
                    all_regressions_incentives['regressions'].append(regression)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")

all_regressions_incentives


Checking date: 2022-04-01 00:00:00, Buffer-1: 2022-03-01 00:00:00 (APR: 0.0), Buffer+1: 2022-05-01 00:00:00 (APR: 0.0)
Checking date: 2022-04-01 00:00:00, Buffer-2: 2022-02-01 00:00:00 (APR: 0.0), Buffer+2: 2022-06-01 00:00:00 (APR: 0.0)
Checking date: 2022-04-01 00:00:00, Buffer-1: 2022-03-01 00:00:00 (APR: 0), Buffer+1: 2022-05-01 00:00:00 (APR: 0)
Checking date: 2022-04-01 00:00:00, Buffer-2: 2022-02-01 00:00:00 (APR: 0), Buffer+2: 2022-06-01 00:00:00 (APR: 0)
[Timestamp('2022-05-01 00:00:00'), Timestamp('2022-06-01 00:00:00'), Timestamp('2022-07-01 00:00:00'), Timestamp('2022-08-01 00:00:00'), Timestamp('2022-09-01 00:00:00')]
Checking date: 2022-08-01 00:00:00, Buffer-1: 2022-07-01 00:00:00 (APR: 0.0), Buffer+1: 2022-09-01 00:00:00 (APR: 0.0)
Checking date: 2022-08-01 00:00:00, Buffer-2: 2022-06-01 00:00:00 (APR: 0.0), Buffer+2: 2022-10-01 00:00:00 (APR: 0.0)
Checking date: 2022-08-01 00:00:00, Buffer-1: 2022-07-01 00:00:00 (APR: 0), Buffer+1: 2022-09-01 00:00:00 (APR: 0)
Checking

{'regressions': [{'banks_for_regression': 'abanca * avg',
   'bank_of_interest_event_series':         date    bank  web_traffic  apr  incentive
   2 2022-02-01  abanca    10.448661  0.0        0.0
   3 2022-03-01  abanca    10.450056  0.0        0.0
   4 2022-04-01  abanca    10.609262  0.0      300.0
   5 2022-05-01  abanca    10.301973  0.0        0.0
   6 2022-06-01  abanca    10.465598  0.0        0.0,
   'control_bank_event_series':         date  web_traffic  apr  incentive
   2 2022-02-01     9.428755    0          0
   3 2022-03-01     9.466777    0          0
   4 2022-04-01     9.449632    0          0
   5 2022-05-01     9.394345    0          0
   6 2022-06-01     9.448440    0          0,
   'bank_of_interest_non_event_series':         date    bank  web_traffic  apr  incentive
   5 2022-05-01  abanca    10.301973  0.0        0.0
   6 2022-06-01  abanca    10.465598  0.0        0.0
   7 2022-07-01  abanca    11.034398  0.0        0.0
   8 2022-08-01  abanca    10.895609  0.0

define a function to get the relevant data from the statsmodel output into a dataframe

In [1088]:
def extract_data_from_sm(model):
    model_dict = {}

    # Extracting coefficients
    coefficients = model.params
    model_dict['coefficients'] = coefficients

    # Extracting standard errors of coefficients
    std_errors = model.bse
    model_dict['standard_errors'] = std_errors

    # Extracting t-values
    t_values = model.tvalues
    model_dict['t_values'] = t_values

    # Extracting p-values
    p_values = model.pvalues
    model_dict['p_values'] = p_values

    # Extracting R-squared
    r_squared = model.rsquared
    model_dict['r_squared'] = r_squared

    return pd.DataFrame(model_dict)

define a function to get control and event periods for each bank out of the dictionary and into a dataframe that be regressed 

also creates the interation terms for the regression

In [1089]:
def get_regression_summary(regression): 
    # Extract the DataFrames
    bank_of_interest_event_series = regression['bank_of_interest_event_series']
    control_bank_event_series = regression['control_bank_event_series']
    bank_of_interest_non_event_series = regression['bank_of_interest_non_event_series']
    control_bank_non_event_series = regression['control_bank_non_event_series']

    # Add a 'type' and 'period' columns to each DataFrame
    bank_of_interest_event_series['type'] = 'bank_of_interest_event'
    control_bank_event_series['type'] = 'control_bank_event'
    bank_of_interest_non_event_series['type'] = 'bank_of_interest_non_event'
    control_bank_non_event_series['type'] = 'control_bank_non_event'

    bank_of_interest_event_series['period'] = 'post'
    control_bank_event_series['period'] = 'post'
    bank_of_interest_non_event_series['period'] = 'pre'
    control_bank_non_event_series['period'] = 'pre'

    # Combine the series into a single DataFrame
    combined_df = pd.concat([
        bank_of_interest_event_series,
        control_bank_event_series,
        bank_of_interest_non_event_series,
        control_bank_non_event_series
    ])

    # Reset index to avoid duplicate indices
    combined_df.reset_index(drop=True, inplace=True)

    # Create dummy variables
    combined_df['treatment'] = combined_df['type'].apply(lambda x: 1 if x in ['bank_of_interest_event', 'bank_of_interest_non_event'] else 0)
    combined_df['post'] = combined_df['period'].apply(lambda x: 1 if x == 'post' else 0)

    # Interaction term: treatment * post
    combined_df['treatment_post'] = combined_df['treatment'] * combined_df['post']

    # Dependent variable
    y = combined_df['web_traffic']

    # Independent variables
    X = combined_df[['treatment', 'post', 'treatment_post']]
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(y, X).fit()

    # Get the summary of the regression
    summary = extract_data_from_sm(model)
    summary['output_title'] = regression['banks_for_regression']
    summary['date_of_event'] = bank_of_interest_event_series.date.median()

    return summary 

### get regrssion outputs for each regression for incentives

In [1090]:
all_regression_outputs = []

for regression in all_regressions_incentives['regressions']:
    if len(regression.keys()) > 0:
        summary = get_regression_summary(regression)
        all_regression_outputs.append(summary)
    else: 
        print('No valid regressions found')
        print(regression)


/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_of_interest_event_series['type'] = 'bank_of_interest_event'
/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_bank_event_series['type'] = 'control_bank_event'
/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:11: SettingWithCopyWarning: 
A value is trying to be s

visualize the regression outputs for each bank

In [1091]:
all_incentive_diff_n_diff = pd.concat(all_regression_outputs)

# create a column from the index 
all_incentive_diff_n_diff['variable_name'] = all_incentive_diff_n_diff.index

all_incentive_diff_n_diff = all_incentive_diff_n_diff.reset_index(drop=True)

all_incentive_diff_n_diff

,coefficients,standard_errors,t_values,p_values,r_squared,output_title,date_of_event,variable_name
0,9.577862,0.081777,117.121744,6.668573e-25,0.917764,abanca * avg,2022-04-01,const
1,1.129534,0.115650,9.766824,3.810367e-08,0.917764,abanca * avg,2022-04-01,treatment
2,-0.140272,0.115650,-1.212903,2.427712e-01,0.917764,abanca * avg,2022-04-01,post
3,-0.112014,0.163554,-0.684876,5.032237e-01,0.917764,abanca * avg,2022-04-01,treatment_post
4,9.441992,0.075859,124.467127,2.522300e-25,0.823495,banc sabadell * avg,2022-08-01,const
5,0.462323,0.107281,4.309450,5.399235e-04,0.823495,banc sabadell * avg,2022-08-01,treatment
6,0.186167,0.107281,1.735321,1.019002e-01,0.823495,banc sabadell * avg,2022-08-01,post
7,0.222252,0.151719,1.464896,1.623225e-01,0.823495,banc sabadell * avg,2022-08-01,treatment_post
8,9.441992,0.039214,240.779330,6.595470e-30,0.964921,banc sabadell * avg,2023-04-01,const
9,0.462323,0.055457,8.336551,3.235887e-07,0.964921,banc sabadell * avg,2023-04-01,treatment


get average statistics

In [1092]:

print(all_incentive_diff_n_diff.coefficients.mean())
print(all_incentive_diff_n_diff.p_values.mean())


2.620712363374471
0.09280119288265165


In [1093]:
all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff['variable_name'] == 'treatment_post', 'coefficients'].mean()

0.17796026246040667

view pertinent statistics

In [1094]:

all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']].to_csv('data/incentive_diff_n_diff.csv')
all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']]


,coefficients,p_values,output_title,date_of_event,r_squared
3,-0.112014,0.503224,abanca * avg,2022-04-01,0.917764
7,0.222252,0.162322,banc sabadell * avg,2022-08-01,0.823495
11,0.327541,0.000713,banc sabadell * avg,2023-04-01,0.964921
15,0.201428,0.079266,santander * avg,2022-05-01,0.931276
19,0.357600,0.013562,santander * avg,2022-08-01,0.930601
23,0.070953,0.340741,santander * avg,2023-09-01,0.969457


### get regrssion outputs for each regression for aprs

In [1095]:
all_regression_outputs_apr = []

for regression in all_regressions_apr['regressions']:
    if len(regression.keys()) > 0:
        summary = get_regression_summary(regression)
        all_regression_outputs_apr.append(summary)
    else: 
        print('No valid regressions found')
        print(regression)


/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_of_interest_event_series['type'] = 'bank_of_interest_event'
/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_bank_event_series['type'] = 'control_bank_event'
/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_17214/4120071802.py:11: SettingWithCopyWarning: 
A value is trying to be s

view results as a dataframe

In [1096]:
all_incentive_diff_n_diff = pd.concat(all_regression_outputs_apr)

# create a column from the index 
all_incentive_diff_n_diff['variable_name'] = all_incentive_diff_n_diff.index

all_incentive_diff_n_diff = all_incentive_diff_n_diff.reset_index(drop=True)

all_incentive_diff_n_diff

,coefficients,standard_errors,t_values,p_values,r_squared,output_title,date_of_event,variable_name
0,9.577862,0.087410,109.573905,1.933635e-24,0.931744,abanca * avg,2023-07-01,const
1,1.129534,0.123617,9.137407,9.490692e-08,0.931744,abanca * avg,2023-07-01,treatment
2,0.252117,0.123617,2.039507,5.826861e-02,0.931744,abanca * avg,2023-07-01,post
3,0.213620,0.174820,1.221941,2.394277e-01,0.931744,abanca * avg,2023-07-01,treatment_post
4,9.441992,0.069726,135.416525,6.553149e-26,0.852129,n26 * avg,2023-07-01,const
5,-0.440168,0.098607,-4.463871,3.917619e-04,0.852129,n26 * avg,2023-07-01,treatment
6,0.387988,0.098607,3.934693,1.184027e-03,0.852129,n26 * avg,2023-07-01,post
7,-0.339168,0.139451,-2.432163,2.712297e-02,0.852129,n26 * avg,2023-07-01,treatment_post
8,9.441992,0.075859,124.467127,2.522300e-25,0.823495,banc sabadell * avg,2022-08-01,const
9,0.462323,0.107281,4.309450,5.399235e-04,0.823495,banc sabadell * avg,2022-08-01,treatment


view average results 

In [1097]:
print(all_incentive_diff_n_diff.coefficients.mean())
print(all_incentive_diff_n_diff.p_values.mean())






2.4098899357476613
0.08333159010631042


view important resuls

In [1098]:

all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']].to_csv('data/apr_diff_n_diff.csv')
all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']]


,coefficients,p_values,output_title,date_of_event,r_squared
3,0.213620,0.239428,abanca * avg,2023-07-01,0.931744
7,-0.339168,0.027123,n26 * avg,2023-07-01,0.852129
11,0.222252,0.162322,banc sabadell * avg,2022-08-01,0.823495
15,0.161221,0.252381,banc sabadell * avg,2023-05-01,0.870560
19,0.067912,0.583388,openbank * avg,2022-04-01,0.613395
23,-0.395086,0.117111,openbank * avg,2023-04-01,0.155655
27,-0.608506,0.009877,openbank * avg,2023-10-01,0.377724
31,0.356635,0.172138,myinvestor * avg,2023-01-01,0.762001
35,-0.568917,0.002910,evobanco * avg,2022-11-01,0.918690
39,-0.099968,0.665418,evobanco * avg,2023-04-01,0.786227


### modify function to take average of all banks not included in the time period specified rather than just ing
